#Appendix

## Set up

In [1]:
!pip install dalex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dalex: filename=dalex-1.6.0-py3-none-any.whl size=1045995 sha256=dec81d418d5d29d52c12b85a4fa77022987ee2cd0de3d069b84d49ce673ada24
  Stored in directory: /root/.cache/pip/wheels/c8/45/19/f5810bf7c5ff9a476ebd89bb5b81a18ffcdf93931d17dbb0c1
Successfully built dalex


In [2]:
import dalex as dx
import xgboost

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [3]:
# import kaggle

# kaggle.api.authenticate()

# kaggle.api.dataset_download_files('The_name_of_the_dataset', path='the_path_you_want_to_download_the_files_to', unzip=True)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Task 1

We have two populations **Blue (privileged)** and **Red (unprivileged)**, with the Blue population being **9 times larger** than the Red population.

Individuals from both populations are requesting to attend XAI training to improve competency in this important area. Number of places is limited. The administrators of the training have decided to give priority to enrolling individuals who may need this training in the future, although unfortunately it is difficult to predict who will benefit.

The decision rule adopted:

1. In the Red group, half of the people will find the skills useful in future and half will not. Administrators randomly allocate 50% of people to training.

| Red | Will use XAI	| Will not use XAI | Total |
| --- | --- | --- | --- |
| Enrolled in training	| 25	| 25	| 50 |
| Not enrolled in training	| 25	| 25	| 50 |
| Total	| 50	| 50	| 100 |

2. In the Blue group, 80% of people will find the training useful in future and 20% will not, although of course it is not known who will find it useful. The administrators have built a predictive model based on user behaviour in predicting for whom it will be useful and whom will not. The model has the following performance:

| Blue | Will use XAI	| Will not use XAI | Total |
| --- | --- | --- | --- |
| Enrolled in training	| 60	| 5	| 65 |
| Not enrolled in training	| 20	| 15	| 35 |
| Total	| 80	| 20	| 100 |

**Task:** Calculate the Demographic parity, equal opportunity and predictive rate parity coefficients for this decision rule.

**Starred task:** How can this decision rule be changed to improve its fairness?

1. **Demografic parity** (Group fairness / STP - Statistical parity / Independence)

Demographic parity is a fairness metric whose goal is to ensure a machine learning model's predictions are independent of membership in a sensitive group. In other words, demographic parity is achieved when the probability of a certain prediction is not dependent on sensitive group membership.

Let $A = a$ be a membership in privileged group and $A = b$ be a membership in unprivileged group. Then the model achieves the demografic parity when the following equality holds

$$P(\widehat{Y} = 1 | A = a) = P(\widehat{Y} = 1 | A = b) \iff \widehat{Y} \perp A.$$

Demographic parity ratio is a ratio of selection rates between smallest and largest groups. Return type is a decimal value. A ratio of 1 means all groups have same selection rate.

Demografic parity ratio for our binary variable $A$ is defined as
$DPR = \frac{P(\widehat{Y} = 1| A = b)}{P(\widehat{Y} = 1| A = a)}$, as $b$ (red) is smaller group and $a$ (blue) is larger one.

Thus, we have

$$DPR = \frac{0.5}{0.65} = \frac{50}{65} = \frac{10}{13} \approx 0.77, $$

since $DP_{A=a} = P(\widehat{Y} = 1| A = a) = 0.65$ and $DP_{A=b} = P(\widehat{Y} = 1| A = b) = 0.5$.

Thus, this decision rule does not achieve demografic parity as $P(\widehat{Y} = 1 | A = a) \neq P(\widehat{Y} = 1 | A = b)$.

2. **Equal oportunity** (TPR - True positive rate / Equal opportunity)

Let $A = a$ be a membership in privileged group and $A = b$ be a membership in unprivileged group. The model achieves equal opportunity, when the following equation holds
$$P(\widehat{Y} = 1 | A = a, Y = 1) = P(\widehat{Y} = 1 | A = b, Y = 1).$$

For our decision rule we have
$$EO_{A=a} = P(\widehat{Y} = 1 | A = a, Y = 1) = TPR_{A=a} = \frac{60}{60+20} = 0.75$$
and
$$EO_{A=b} = P(\widehat{Y} = 1 | A = b, Y = 1) = TPR_{A=b} = \frac{25}{25+25} = 0.5$$

Consequently, this decision rule does not present equal opportunity since $P(\widehat{Y} = 1 | A = a, Y = 1) \neq P(\widehat{Y} = 1 | A = b, Y = 1)$

3. **Predictive rate parity** (Predictive Rate Parity, Sufficiency)

Let $A = a$ be a membership in privileged group and $A = b$ be a membership in unprivileged group. The model achieves predictive rate parity, when the following holds
$$P(Y = 1 | A = a, \widehat{Y} = 1) = P(Y = 1 | A = b, \widehat{Y} = 1) \land P(Y = 0 | A = a, \widehat{Y} = 0) = P(Y = 0 | A = b, \widehat{Y} = 0) \iff Y \perp A | \widehat{Y},$$ where $A = a$ is membership in privileged group and $A = b$ is membership in unprivileged group.

For our decision rule we have
$$P(Y = 1 | A = a, \widehat{Y} = 1) = \frac{TP}{TP+FP} = \frac{60}{60 + 5} = \frac{12}{13} \approx 0.92,$$
$$P(Y = 1 | A = b, \widehat{Y} = 1) = \frac{TP}{TP+FP} = \frac{25}{25 + 25} = 0.5,$$
$$P(Y = 0 | A = a, \widehat{Y} = 0) = \frac{TN}{TN + FN} = \frac{15}{15 + 20} = \frac{3}{7} \approx 0.43,$$
$$P(Y = 0 | A = b, \widehat{Y} = 0) = \frac{TN}{TN + FN} =\frac{25}{25 + 25} = 0.5.$$

Thus the model does not achieve predictive rate parity as $P(Y = 1 | A = a, \widehat{Y} = 1) \neq P(Y = 1 | A = b, \widehat{Y} = 1)$ as well as $P(Y = 0 | A = a, \widehat{Y} = 0) \neq P(Y = 0 | A = b, \widehat{Y} = 0)$.

# Task 2

## Dataset

We selected [adult income](https://www.kaggle.com/datasets/wenruliu/adult-income-dataset).  The datasets consists of 14 predictor variables including protected attributes such as age, gender, race and predicted variable income that is divided into two classes <=50K and >50K. For further analysis we will be focusing on gender as our selected protected attribute.

### Loading dataset

In [5]:
PATH = '/content/drive/My Drive/Colab Notebooks/EML/HW2/income/adult.csv'
df = pd.read_csv(PATH)
df

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [7]:
print(df["race"].unique())
print(df["age"].unique())
print(df["gender"].unique())

['Black' 'White' 'Asian-Pac-Islander' 'Other' 'Amer-Indian-Eskimo']
[25 38 28 44 18 34 29 63 24 55 65 36 26 58 48 43 20 37 40 72 45 22 23 54
 32 46 56 17 39 52 21 42 33 30 47 41 19 69 50 31 59 49 51 27 57 61 64 79
 73 53 77 80 62 35 68 66 75 60 67 71 70 90 81 74 78 82 83 85 76 84 89 88
 87 86]
['Male' 'Female']


In [8]:
df["income"].unique()

array(['<=50K', '>50K'], dtype=object)

Check if any NaN values. Replacing NaN values is necessary only for `sklearn.GradientBoostingClassifier` not `sklearn.HistGradientBoostingClassifier`.

In [9]:
df.isnull().values.any()

False

### Change object data types to categorical.

Not actually necessary as we still need to encode the catgories as floats or ints.

In [10]:
df.loc[:, df.dtypes == 'object'] =\
    df.select_dtypes(['object'])\
    .apply(lambda x: x.astype('category'))

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   age              48842 non-null  int64   
 1   workclass        48842 non-null  category
 2   fnlwgt           48842 non-null  int64   
 3   education        48842 non-null  category
 4   educational-num  48842 non-null  int64   
 5   marital-status   48842 non-null  category
 6   occupation       48842 non-null  category
 7   relationship     48842 non-null  category
 8   race             48842 non-null  category
 9   gender           48842 non-null  category
 10  capital-gain     48842 non-null  int64   
 11  capital-loss     48842 non-null  int64   
 12  hours-per-week   48842 non-null  int64   
 13  native-country   48842 non-null  category
 14  income           48842 non-null  category
dtypes: category(9), int64(6)
memory usage: 2.7 MB


In [30]:
# save catgorical columns names
categorical_columns = df.select_dtypes(include=['category']).columns.to_list()
categorical_columns.remove("income")
numerical_columns = df.select_dtypes(include=['int64']).columns.to_list()

### Original encoding for categorical variables

One hot encoding is only necessary for `sklearn.GradientBoostingClassifier`.
Since `sklearn.HistGradientBoostingClassifier` can use categorical variables, but first they must be encoded, however it does not need to be one-hot-encoding.

In [67]:
ordinal_encoder = make_column_transformer(
    (
        OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan),
        make_column_selector(dtype_include="category"),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
)

# transformed = ordinal_encoder.fit_transform(df)
ordinal_encoder.fit(df)
transformed = ordinal_encoder.transform(df)
transformed_df = pd.DataFrame(transformed, columns=ordinal_encoder.get_feature_names_out())
transformed_df

,workclass,education,marital-status,occupation,relationship,race,gender,native-country,income,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
0,4.0,1.0,4.0,7.0,3.0,2.0,1.0,39.0,0.0,25.0,226802.0,7.0,0.0,0.0,40.0
1,4.0,11.0,2.0,5.0,0.0,4.0,1.0,39.0,0.0,38.0,89814.0,9.0,0.0,0.0,50.0
2,2.0,7.0,2.0,11.0,0.0,4.0,1.0,39.0,1.0,28.0,336951.0,12.0,0.0,0.0,40.0
3,4.0,15.0,2.0,7.0,0.0,2.0,1.0,39.0,1.0,44.0,160323.0,10.0,7688.0,0.0,40.0
4,0.0,15.0,4.0,0.0,3.0,4.0,0.0,39.0,0.0,18.0,103497.0,10.0,0.0,0.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,4.0,7.0,2.0,13.0,5.0,4.0,0.0,39.0,0.0,27.0,257302.0,12.0,0.0,0.0,38.0
48838,4.0,11.0,2.0,7.0,0.0,4.0,1.0,39.0,1.0,40.0,154374.0,9.0,0.0,0.0,40.0
48839,4.0,11.0,6.0,1.0,4.0,4.0,0.0,39.0,0.0,58.0,151910.0,9.0,0.0,0.0,40.0
48840,4.0,11.0,4.0,1.0,3.0,4.0,1.0,39.0,0.0,22.0,201490.0,9.0,0.0,0.0,20.0


In [28]:
transformed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   workclass        48842 non-null  float64
 1   education        48842 non-null  float64
 2   marital-status   48842 non-null  float64
 3   occupation       48842 non-null  float64
 4   relationship     48842 non-null  float64
 5   race             48842 non-null  float64
 6   gender           48842 non-null  float64
 7   native-country   48842 non-null  float64
 8   income           48842 non-null  float64
 9   age              48842 non-null  float64
 10  fnlwgt           48842 non-null  float64
 11  educational-num  48842 non-null  float64
 12  capital-gain     48842 non-null  float64
 13  capital-loss     48842 non-null  float64
 14  hours-per-week   48842 non-null  float64
dtypes: float64(15)
memory usage: 5.6 MB


In [23]:
X = transformed_df.drop(columns = "income")
y = transformed_df.income

### Train-test split

In [29]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.33, random_state=42)

## Model Ia - with protected attribute

We use `sklearn.HistGradientBoostingClassifier` from as it:
* has built-in support for missing values (NaNs).
* has Categorical Features Support
* is also reportedly faster than `sklearn.GradientBoostingClassifier`.

In [33]:
model = HistGradientBoostingClassifier(learning_rate=0.1,
                                       max_iter = 100,
                                       max_depth=None,
                                       max_bins=255,
                                       categorical_features=categorical_columns,
                                       random_state=42,
                                       )

In [34]:
model.fit(X_train, y_train)

HistGradientBoostingClassifier(categorical_features=['workclass', 'education',
                                                     'marital-status',
                                                     'occupation',
                                                     'relationship', 'race',
                                                     'gender',
                                                     'native-country'],
                               random_state=42)

In [174]:
def pf_histboost_classifier_categorical(model, df):
    df.loc[:, df.dtypes == 'object'] =\
        df.select_dtypes(['object'])\
        .apply(lambda x: x.astype('category'))
    return model.predict_proba(df)[:, 1]

explainer = dx.Explainer(model, X_test, y_test, predict_function=pf_histboost_classifier_categorical, label="HistGBClassifier with protected attribute")

Preparation of a new explainer is initiated

  -> data              : 16118 rows 14 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 16118 values
  -> model_class       : sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier (default)
  -> label             : HistGBClassifier with protected attribute
  -> predict function  : <function pf_histboost_classifier_categorical at 0x7da0ef360160> will be used
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.000175, mean = 0.238, max = 0.999
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.997, mean = -0.00198, max = 0.996
  -> model_info        : package sklearn

A new explainer has been created!


### Evaluation

In [175]:
explainer.model_performance()

,recall,precision,f1,accuracy,auc
HistGBClassifier with protected attribute,0.660442,0.788383,0.718763,0.878087,0.931761


In [176]:
explainer.model_parts().result

,variable,dropout_loss,label
0,_full_model_,0.065989,HistGBClassifier with protected attribute
1,gender,0.066769,HistGBClassifier with protected attribute
2,race,0.066862,HistGBClassifier with protected attribute
3,fnlwgt,0.067056,HistGBClassifier with protected attribute
4,native-country,0.067687,HistGBClassifier with protected attribute
5,educational-num,0.067731,HistGBClassifier with protected attribute
6,workclass,0.068181,HistGBClassifier with protected attribute
7,relationship,0.076109,HistGBClassifier with protected attribute
8,hours-per-week,0.078460,HistGBClassifier with protected attribute
9,capital-loss,0.079866,HistGBClassifier with protected attribute


### Fairness coefficients

We select protected variable "gender" and privileded group "Male".

We computed Statistical parity (STP), Equal opportunity/True positive rate (TPR), Predictive parity/Positive predicitive value (PPV) for our model predictions. The values were equal to:

$$STP = 0.286822$$
$$TRP = 0.840237$$
$$PPV = 1.036943$$

In [177]:
protected_variable = X_test.gender.apply(lambda x: "Male" if x else "Female")
privileged_group = "Male"

fobject = explainer.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)

In [178]:
fobject.fairness_check()

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'Male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
Female  0.840237  1.108491  1.036943  0.189873  0.286822


In [179]:
fobject.plot()

## Model Ib - without protected attribute


In [180]:
X_train_without_prot, X_test_without_prot = X_train.drop("gender", axis=1), X_test.drop("gender", axis=1)
categorical_columns_without_prot = categorical_columns.copy()
categorical_columns_without_prot.remove("gender")

In [181]:
model_without_prot = HistGradientBoostingClassifier(learning_rate=0.1,
                                       max_iter = 100,
                                       max_depth=None,
                                       max_bins=255,
                                       categorical_features=categorical_columns_without_prot,
                                       random_state=42,
                                       )

In [182]:
model_without_prot.fit(X_train_without_prot, y_train)

HistGradientBoostingClassifier(categorical_features=['workclass', 'education',
                                                     'marital-status',
                                                     'occupation',
                                                     'relationship', 'race',
                                                     'native-country'],
                               random_state=42)

In [183]:
explainer_without_prot = dx.Explainer(model_without_prot, X_test_without_prot, y_test, predict_function=pf_histboost_classifier_categorical, label="HistGBClassifier without protected attribute")

Preparation of a new explainer is initiated

  -> data              : 16118 rows 13 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 16118 values
  -> model_class       : sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier (default)
  -> label             : HistGBClassifier without protected attribute
  -> predict function  : <function pf_histboost_classifier_categorical at 0x7da0ef360160> will be used
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.000169, mean = 0.238, max = 0.999
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.998, mean = -0.00224, max = 0.996
  -> model_info        : package sklearn

A new explainer has been created!


In [206]:
pd.concat([
    explainer.model_performance().result,
    explainer_without_prot.model_performance().result,
], axis=0)

,recall,precision,f1,accuracy,auc
HistGBClassifier with protected attribute,0.660442,0.788383,0.718763,0.878087,0.931761
HistGBClassifier without protected attribute,0.664387,0.786181,0.720171,0.878211,0.931389


In [186]:
protected_variable = X_test.gender.apply(lambda x: "Male" if x else "Female")
privileged_group = "Male"

fobject_without_prot = explainer_without_prot.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)

In [187]:
fobject.fairness_check()
print("\n")
fobject_without_prot.fairness_check()

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'Male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
Female  0.840237  1.108491  1.036943  0.189873  0.286822


Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'Male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV  FPR       STP
Female  0.841176  1.108491  1.026786  0.2  0.288462


In [188]:
fobject.plot(fobject_without_prot, show=False)

## Model II with protected attribute

Since the model's performance with and without protected attribute is comparable and fairness metrics for further experiments, we will be using the model with protected attribute for further experiments.

In [189]:
model_2 = HistGradientBoostingClassifier(learning_rate=0.1,
                                       max_iter = 100,
                                       max_depth=3,
                                       max_bins=255,
                                       categorical_features=categorical_columns,
                                       random_state=42,
                                       )

In [190]:
model_2.fit(X_train, y_train)

HistGradientBoostingClassifier(categorical_features=['workclass', 'education',
                                                     'marital-status',
                                                     'occupation',
                                                     'relationship', 'race',
                                                     'gender',
                                                     'native-country'],
                               max_depth=3, random_state=42)

In [191]:
explainer_2 = dx.Explainer(model_2, X_test, y_test, predict_function=pf_histboost_classifier_categorical, label="HistGBClassifier with protected attribute 2")

Preparation of a new explainer is initiated

  -> data              : 16118 rows 14 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 16118 values
  -> model_class       : sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier (default)
  -> label             : HistGBClassifier with protected attribute 2
  -> predict function  : <function pf_histboost_classifier_categorical at 0x7da0ef360160> will be used
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.000582, mean = 0.238, max = 0.997
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.985, mean = -0.00209, max = 0.994
  -> model_info        : package sklearn

A new explainer has been created!


In [205]:
pd.concat([
    explainer.model_performance().result,
    explainer_2.model_performance().result,
], axis=0)

,recall,precision,f1,accuracy,auc
HistGBClassifier with protected attribute,0.660442,0.788383,0.718763,0.878087,0.931761
HistGBClassifier with protected attribute 2,0.633614,0.796627,0.705831,0.875419,0.926327


In [194]:
protected_variable = X_test.gender.apply(lambda x: "Male" if x else "Female")
privileged_group = "Male"

fobject_2 = explainer_2.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)

In [195]:
fobject.fairness_check()
print("\n")
fobject_2.fairness_check()

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'Male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
Female  0.840237  1.108491  1.036943  0.189873  0.286822


Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'Male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
Female  0.768293  1.110059  1.068268  0.148649  0.253012


In [196]:
fobject.plot(fobject_2, show=False)

## Bias mitigation technique - selcting the best technique.

For Model 1a we performed the following bias mitigation methods:

* **resample** - returns indices used to pick relevant samples of data
* **reweight** - returns sample (case) weights for model training
* **roc_pivot** - returns the `Explainer` with a changed `y_hat` prediction


In [197]:
from dalex.fairness import resample, reweight, roc_pivot
from copy import copy

In [198]:
from dalex.fairness import resample, reweight, roc_pivot
from copy import copy

protected_variable_train = X_train.gender.apply(lambda x: "Male" if x else "Female")

# resample
indices_resample = resample(
    protected_variable_train,
    y_train,
    type='preferential', # uniform
    probs=model.predict_proba(X_train)[:, 1], # requires probabilities
    verbose=False
)
model_resample = copy(model)
model_resample.fit(X_train.iloc[indices_resample, :], y_train.iloc[indices_resample])
explainer_resample = dx.Explainer(
    model_resample,
    X_test,
    y_test,
    label='HistGBClassifier with Resample mitigation',
    verbose=False
)
fobject_resample = explainer_resample.model_fairness(
    protected_variable,
    privileged_group
)

# reweight
sample_weight = reweight(
    protected_variable_train,
    y_train,
    verbose=False
)
model_reweight = copy(model)
model_reweight.fit(X_train, y_train, sample_weight=sample_weight)
explainer_reweight = dx.Explainer(
    model_reweight,
    X_test,
    y_test,
    label='HistGBClassifier with Reweight mitigation',
    verbose=False
)
fobject_reweight = explainer_reweight.model_fairness(
    protected_variable,
    privileged_group
)

# roc_pivot
explainer_roc_pivot = roc_pivot(
    copy(explainer),
    protected_variable,
    privileged_group,
    verbose=False
)
explainer_roc_pivot.label = 'HistGBClassifier with ROC pivot mitigation'
fobject_roc_pivot = explainer_roc_pivot.model_fairness(
    protected_variable,
    privileged_group
)

In [199]:
fobject.plot([fobject_resample, fobject_reweight, fobject_roc_pivot], show=False)

In [200]:
for fobj in [fobject, fobject_resample, fobject_reweight, fobject_roc_pivot]:
    print("\n========== " + fobj.label + " ==========")
    fobj.fairness_check(epsilon=0.8)


========== HistGBClassifier with protected attribute ==========
Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'Male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
Female  0.840237  1.108491  1.036943  0.189873  0.286822

========== HistGBClassifier with Resample mitigation ==========
Bias detected in 3 metrics: TPR, PPV, FPR

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'Male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
Female  1.569091  1.050119  0.542491  3.025641  1.026042

========== HistGBClassifier with Reweight mitigation ==========
Bias detected in 2 metrics: PPV, STP

Co

To analyze the fairness metrics of the models, the epsilon was set to 0.8 according to the "four-fifths rule".
We can observe that the resample bias mitigation method actually worsened the bias of the model since TPR, PPV, and FPR metrics exceeded the acceptable limit set by epsilon.

The ROC pivot mitigation slightly increased the values of FPR and STP metrics, however, they still were smaller than the lower bound of the acceptable interval (0.8, 1.25).

The best model with respect to fairness metrics seems to be the HistGBClassifier with reweight mitigation since it has only two metrics: PPV, and STP outside of acceptable interval, however PPV = 0.792727, which is only slightly lower than acceptable 0.8. Even though STP for this model is not as high as after resampling, where this metric reaches the highest observed value of 1.026042, it is closer to the epsilon than in the model without any mitigation technique.

Experimenting with different bias mitigation techniques shows that for each of them, we need to accept some tradeoff between fairness metrics. After further inspection of metrics values one might suggest that in this case reweight mitigation would be the best technique to use.


In [201]:
pd.concat([
    explainer.model_performance().result,
    explainer_resample.model_performance().result,
    explainer_reweight.model_performance().result,
    explainer_roc_pivot.model_performance().result
], axis=0)

,recall,precision,f1,accuracy,auc
HistGBClassifier with protected attribute,0.660442,0.788383,0.718763,0.878087,0.931761
HistGBClassifier with Resample mitigation,0.596791,0.727477,0.655686,0.852153,0.900495
HistGBClassifier with Reweight mitigation,0.626249,0.789456,0.698445,0.872441,0.927837
HistGBClassifier with ROC pivot mitigation,0.633877,0.806830,0.709972,0.877838,0.931678


The comparison of mitigation techniques shows that ** mitigation** method shows
* no significant drop in the following metrics: accuracy, auc, and f1,
* drop in recall metric,
* rise in precision metric.

However, this technique showed smaller overall fairness improvement than **reweight mitigation** which showed:
* no significant drop in precision, accuracy, and AUC metrics
* slight drop in f1 metric and recall.

Those results confirm that HistGBClassifier with Reweight mitigation seems to be the optimal model.


## Comparing Model 1a, Model 2 and HistGBClassifier with protected attribite

In [202]:
fobject.plot([fobject_2, fobject_reweight], show=False)

In [203]:
for fobj in [fobject, fobject_2, fobject_reweight]:
    print("\n========== " + fobj.label + " ==========")
    fobj.fairness_check(epsilon=0.8)


========== HistGBClassifier with protected attribute ==========
Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'Male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
Female  0.840237  1.108491  1.036943  0.189873  0.286822

========== HistGBClassifier with protected attribute 2 ==========
Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'Male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
Female  0.768293  1.110059  1.068268  0.148649  0.253012

========== HistGBClassifier with Reweight mitigation ==========
Bias detected in 2 metrics: PPV, STP



In [204]:
pd.concat([
    explainer.model_performance().result,
    explainer_2.model_performance().result,
    explainer_reweight.model_performance().result,
], axis=0)

,recall,precision,f1,accuracy,auc
HistGBClassifier with protected attribute,0.660442,0.788383,0.718763,0.878087,0.931761
HistGBClassifier with protected attribute 2,0.633614,0.796627,0.705831,0.875419,0.926327
HistGBClassifier with Reweight mitigation,0.626249,0.789456,0.698445,0.872441,0.927837


This comparison shows similar drop in performance for Model 2 and after reweight mitigation in comparison to Model 1a performance. The fainess coefficient for Model 1a and Model 2 are similar while reweight mitigation increased values of all metrics except for predicitive parity ratio (PPV).